In [1]:
import cv2 
  
# path 
path = r'w frac.png'
  
# Reading an image in grayscale mode 
image = cv2.imread(path, 0) 
image = cv2.bitwise_not(image)
# Window name in which image is displayed 
window_name = 'image'
  
# Using cv2.imshow() method 
# Displaying the image 
cv2.imshow(window_name, image) 
  
# waits for user to press any key 
# (this is necessary to avoid Python kernel form crashing) 
cv2.waitKey(0) 
  
# closing all open windows 
cv2.destroyAllWindows()

In [2]:
bb_info_list = []

min_contour_area = 100
image_with_rectangles = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

import cv2 
contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
image_with_rectangles = image.copy()
image_with_rectangles = cv2.cvtColor(image_with_rectangles, cv2.COLOR_GRAY2BGR)


from PIL import Image 
# get image 
img = Image.open(path) 
  
# get width and height 
width = img.width 
height = img.height

for contour in contours:
    contour_area = cv2.contourArea(contour)
    if contour_area >= min_contour_area:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image_with_rectangles, (x, y), (x + w, y + h), (0, 0, 255), 1)
        [xmin, xmax, ymin, ymax,  img_width, img_height] = x, (x+w), y, (y+h), img.width, img.height 
        bb_info_list.append([xmin, xmax, ymin, ymax,  img_width, img_height])

#To change pathname to {class}.jpg
cv2.imwrite('/content/hpboundrec.jpg', image_with_rectangles)
cv2.imshow(window_name,image_with_rectangles)
cv2.waitKey(0) 

-1

In [4]:
bb_info_list

[[15, 56, 144, 198, 256, 256],
 [3, 72, 127, 136, 256, 256],
 [160, 193, 107, 144, 256, 256],
 [97, 144, 94, 146, 256, 256],
 [32, 46, 84, 123, 256, 256],
 [186, 217, 74, 105, 256, 256],
 [218, 253, 59, 147, 256, 256]]